In [ ]:
import os
import pandas as pd
import sys

# project_path = "."
project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "Cariplo_NRe/IEGs Experiment", True
DATA_ROOT, PLOTS_ROOT = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, False, "")
data_input_path = os.path.join(DATA_ROOT, "BraiAn_output")

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
brain_onthology = BraiAn.AllenBrainHierarchy(path_to_allen_json, ["fiber tracts", "VS", "grv", "retina", "CB"], version="ccfv3")

In [ ]:
REGIONS_TO_PLOT_SELECTION_METHOD = "structural level 8"
# REGIONS_TO_PLOT_SELECTION_METHOD = "major divisions"
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        brain_onthology.select_from_csv(path_to_summary_structures)
    case "major divisions":
        brain_onthology.select_regions(BraiAn.MAJOR_DIVISIONS)
    case "smallest":
        brain_onthology.select_leaves()
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        brain_onthology.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        brain_onthology.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")
selected_regions = brain_onthology.get_selected_regions()
print(f"You selected {len(selected_regions)} regions to plot.")

In [ ]:
group_1: BraiAn.AnimalGroup = BraiAn.AnimalGroup.from_csv("CTX", data_input_path, f"cell_counts_CTX_density.csv")
group_2: BraiAn.AnimalGroup = BraiAn.AnimalGroup.from_csv("FC", data_input_path, f"cell_counts_FC_density.csv")
if not group_1.is_comparable(group_2):
    raise ImportError("Group 1 and Group 2 are not comparable!\n\
Please check that you're reading two groups that normalized on the same brain regions and on the same marker")

In [ ]:
# pls = group_1.pls_regions(group_2, selected_regions, n_permutations=5000, n_bootstrap=5000)

In [ ]:
group_3 = BraiAn.AnimalGroup("G2", group_2.animals[:3], group_2.metric, False, None, False)
group_4 = BraiAn.AnimalGroup("G3", group_2.animals[3:], group_2.metric, False, None, False)

In [ ]:
BraiAn.plot_gridgroups([group_1, group_3, group_4],
                       selected_regions,
                       "cFos",
                       brain_onthology=brain_onthology,
                       pls_n_permutations=5000, pls_n_bootstrap=5000,
                       height=5000, width=None,
                       barplot_width=0.8, space_between_markers=0.01,
                       groups_marker1_colours=["LightCoral", "SandyBrown", "green"],
                       groups_marker2_colours=["IndianRed", "Orange", "lightgreen"],
                       color_heatmap="deep_r")

In [ ]:
BraiAn.plot_gridgroups([group_1, group_3, group_4],
# BraiAn.plot_gridgroups([group_1, group_2],
                       selected_regions,
                       # list(set(selected_regions)),
                       "cFos", "Arc1",
                       brain_onthology=brain_onthology,
                       # brain_onthology=None,
                       pls_n_permutations=5000, pls_n_bootstrap=5000,
                       height=5000, width=None,
                       barplot_width=0.7, space_between_markers=0.01,
                       groups_marker1_colours=["LightCoral", "SandyBrown", "green"],
                       groups_marker2_colours=["IndianRed", "Orange", "lightgreen"],
                       color_heatmap="deep_r")